
# https://ieeexplore.ieee.org/abstract/document/8682391


# synchro squeezing of stft

In [ ]:
import os
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal, ndimage
import random
from scipy.interpolate import interp1d

# Function to convert a single text file to CSV with headers and set the "Activity" value
def txt_to_csv(input_file, output_file):
    activity_mapping = {
        "Bowing": 1,
        "Clapping": 1, 
        "Handshaking": 1,
        "Hugging": 1,
        "Jumping": 1,
        "Running": 1,
        "Seating": 1,
        "Standing": 1,
        "Walking": 1,
        "Waving": 1
    }
    default_activity = 0

    headers = ["R-Bic", "R-Tri", "L-Bic", "L-Tri", "R-Thi", "R-Ham", "L-Thi", "L-Ham", "Activity"]

    file_name = os.path.splitext(os.path.basename(input_file))[0]
    activity = activity_mapping.get(file_name, default_activity)

    with open(input_file, 'r') as txt_file, open(output_file, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(headers)
        for line in txt_file:
            row = line.strip().split('\t')
            row.append(activity)
            csv_writer.writerow(row)

# Function to convert .txt files to .csv with headers and set the "Activity" value
def convert_txt_to_csv_in_subfolders(root_directory, output_ssst_directory, sample_and_create_ssst_func):
    for subfolder in os.listdir(root_directory):
        subfolder_path = os.path.join(root_directory, subfolder)
        if os.path.isdir(subfolder_path):
            # Check if the subfolder is a subject folder
            if subfolder.startswith('sub'):  # Adjust the condition based on your subject folder naming convention
                subject_number = subfolder.split('sub')[1]  # Extract subject number from folder name
                for filename in os.listdir(subfolder_path):
                    if filename.endswith('.txt'):
                        txt_file_path = os.path.join(subfolder_path, filename)
                        csv_file_path = os.path.splitext(txt_file_path)[0] + '.csv'
                        txt_to_csv(txt_file_path, csv_file_path)
                        print(f"Converted: {txt_file_path} to {csv_file_path}")

                        # Sampling and SS-STFT creation
                        sample_and_create_ssst_func(csv_file_path, output_ssst_directory, subject_number)

# Function to sample CSV data randomly and create Synchrosqueezing Transform on STFT (SS-STFT)
def sample_and_create_ssst_random(csv_file_path, output_directory, subject_number):
    df = pd.read_csv(csv_file_path)
    activity = df['Activity'].iloc[0]
    sensor_columns = ["R-Bic", "R-Tri", "L-Bic", "L-Tri", "R-Thi", "R-Ham", "L-Thi", "L-Ham"]

    sample_size = 500
    num_samples = df.shape[0] // sample_size
    num_segments_per_channel = 30
    file_name = os.path.splitext(os.path.basename(csv_file_path))[0]

    for sensor in sensor_columns:
        for i in range(num_segments_per_channel):
            # Randomly select a start index within the valid range
            start_index = random.randint(0, num_samples - 1) * sample_size
            end_index = start_index + sample_size
            segment = df.loc[start_index:end_index - 1, sensor].values

            f, t, Zxx = signal.stft(segment, fs=1.0)  # Compute STFT
            omega, _, _ = signal.spectrogram(segment, fs=1.0)  # Compute Instantaneous Frequencies

            # Ensure omega has the same length as f
            if len(omega) < len(f):
                f_interp = interp1d(np.arange(len(omega)), omega, kind='linear')
                omega = f_interp(np.linspace(0, len(omega) - 1, len(f)))

            # Apply Synchrosqueezing Transform to STFT
    
            omega_matrix = omega.reshape(-1, 1)

            # Check for zero values in omega and replace with a small non-zero value
            zero_indices = np.where(omega_matrix == 0)
            omega_matrix[zero_indices] = 1e-10

            ssst_matrix = ndimage.sobel(np.angle(Zxx / omega_matrix), axis=0)


            # Plot SS-STFT without labels
            plt.figure(figsize=(5,5))
            plt.pcolormesh(t, f, np.abs(ssst_matrix))  # Magnitude of SS-STFT
            plt.axis('off')  # Turn off axis labels and ticks
            plt.gca().set_aspect('auto')  # Adjust aspect ratio for better visualization

            # Save SS-STFT images in time-frequency format in activity-wise subfolders
            activity_output_directory = os.path.join(output_directory, f"Activity_{activity}")
            os.makedirs(activity_output_directory, exist_ok=True)

            ssst_image_path = os.path.join(activity_output_directory, f"SSST_Sub{subject_number}{file_name}_sensor{sensor}_random{i + 1}.png")
            plt.savefig(ssst_image_path, bbox_inches='tight', pad_inches=0, transparent=True)
            plt.close()
            print(f"Saved SS-STFT image: {ssst_image_path}")

# Specify the root directory containing the subfolders and the directory to save SS-STFT images
root_directory = 'D:/EMG Physical Action Data Set'
output_ssst_directory = 'C:/Users/saisr/OneDrive/Desktop/ssst_images_output'  # Change this path

# Call the function to convert .txt files to .csv with headers and set the "Activity" value
convert_txt_to_csv_in_subfolders(root_directory, output_ssst_directory, sample_and_create_ssst_random)


In [ ]:
import os
import shutil

# Root directory containing the 'Activity' folders
root_directory = 'C:/Users/saisr/OneDrive/Desktop/ssst_images_output'

# Define the range of activity folders, e.g., 0 to 10
activity_range = range(11)  # This will cover Activity_0 to Activity_10

for activity_number in activity_range:
    activity_folder = f'Activity_{activity_number}'
    activity_folder_path = os.path.join(root_directory, activity_folder)
    
    # Check if the Activity folder exists and if it's a directory
    if os.path.exists(activity_folder_path) and os.path.isdir(activity_folder_path):
        sub_activity_folder = f'{activity_folder}.0'
        sub_activity_folder_path = os.path.join(root_directory, sub_activity_folder)
        
        # Check if the subfolder Activity_0.0 exists and is a directory
        if os.path.exists(sub_activity_folder_path) and os.path.isdir(sub_activity_folder_path):
            # Move contents from 'Activity_0.0' to 'Activity_0'
            for item in os.listdir(sub_activity_folder_path):
                source = os.path.join(sub_activity_folder_path, item)
                destination = os.path.join(activity_folder_path, item)
                shutil.move(source, destination)
            
            # Remove the now-empty 'Activity_0.0' folder
            os.rmdir(sub_activity_folder_path)
            print(f"Moved contents of {sub_activity_folder} to {activity_folder} and deleted {sub_activity_folder}")
